<a href="https://www.kaggle.com/code/shashankfab/newnb?scriptVersionId=110993263" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
label_file_name = "../input/dog-breed-identification-mlbootcamp/labels.csv"

training_set = pd.read_csv(label_file_name)

In [3]:
training_set

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [4]:
training_imgs = ["../input/dog-breed-identification-mlbootcamp/train/{}.jpg".format(x) for x in list(training_set.id)]

training_labels_2 = list(training_set['breed'])
training_set = pd.DataFrame( {'Images': training_imgs,'Breed' : training_labels_2})

#Changing the type  to str
training_set.Animal = training_set.Images.astype(str)
training_set.Breed = training_set.Breed.astype(str)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys


In [5]:
training_set

,Images,Breed
0,../input/dog-breed-identification-mlbootcamp/t...,boston_bull
1,../input/dog-breed-identification-mlbootcamp/t...,dingo
2,../input/dog-breed-identification-mlbootcamp/t...,pekinese
3,../input/dog-breed-identification-mlbootcamp/t...,bluetick
4,../input/dog-breed-identification-mlbootcamp/t...,golden_retriever
...,...,...
10217,../input/dog-breed-identification-mlbootcamp/t...,borzoi
10218,../input/dog-breed-identification-mlbootcamp/t...,dandie_dinmont
10219,../input/dog-breed-identification-mlbootcamp/t...,airedale
10220,../input/dog-breed-identification-mlbootcamp/t...,miniature_pinscher


In [6]:
pixels_x = 250
pixels_y = 250
channels = 3

target_size=(pixels_x,pixels_y)
input_shape_size=(pixels_x,pixels_y,channels)

no_of_epochs_1 = 100
no_of_epochs_2 = 100

In [7]:


train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  validation_split= 0.2,
                                  horizontal_flip = True
                                  )

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="",x_col="Images",
                                        y_col="Breed",
                                        class_mode="categorical",
                                        target_size=target_size,
                                        batch_size=32,
                                        subset='training',
                                        validate_filenames=False)

val_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="",x_col="Images",
                                        y_col="Breed",
                                        class_mode="categorical",
                                        target_size=target_size,
                                        batch_size=32,
                                        subset='validation',
                                        validate_filenames=False)

Found 8178 non-validated image filenames belonging to 120 classes.
Found 2044 non-validated image filenames belonging to 120 classes.


In [8]:
model = Sequential([    
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    
    Flatten(),
    
    Dense(units=1200,activation="relu"),
    Dropout(0.4),
    Dense(units=400,activation="relu"),
    Dropout(0.2),
    Dense(units=360,activation="relu"),
    Dropout(0.2),
    Dense(units=240,activation="relu"),
    Dense(units=120,activation="softmax")
])

2022-11-14 17:56:24.693702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 17:56:24.694639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 17:56:24.978870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 17:56:24.979807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 17:56:24.980713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [9]:
model.compile(
        optimizer='rmsprop',
        loss="categorical_crossentropy",
        metrics=["accuracy"]
)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8,
    min_delta=0.001
)

In [11]:
model.fit_generator(
    train_generator, 
    epochs = no_of_epochs_1, 
    steps_per_epoch = 60, 
    validation_data= val_generator , 
    shuffle= True,
    callbacks=[custom_early_stopping]
)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-14 17:56:33.553228: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-11-14 17:56:44.477958: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
60/60 [==============================] - 96s 2s/step - loss: 5.8901 - accuracy: 0.0094 - val_loss: 4.7842 - val_accuracy: 0.0083
Epoch 2/100
60/60 [==============================] - 77s 1s/step - loss: 4.7967 - accuracy: 0.0173 - val_loss: 4.7861 - val_accuracy: 0.0103
Epoch 3/100
60/60 [==============================] - 75s 1s/step - loss: 4.7858 - accuracy: 0.0130 - val_loss: 4.7924 - val_accuracy: 0.0098
Epoch 4/100
60/60 [==============================] - 73s 1s/step - loss: 4.7891 - accuracy: 0.0115 - val_loss: 4.7858 - val_accuracy: 0.0113
Epoch 5/100
60/60 [==============================] - 72s 1s/step - loss: 4.7917 - accuracy: 0.0094 - val_loss: 4.7883 - val_accuracy: 0.0127
Epoch 6/100
60/60 [==============================] - 72s 1s/step - loss: 4.7830 - accuracy: 0.0156 - val_loss: 4.7717 - val_accuracy: 0.0176
Epoch 7/100
60/60 [==============================] - 73s 1s/step - loss: 4.7716 - accuracy: 0.0156 - val_loss: 4.7484 - val_accuracy: 0.0191
Epoch 8/100
6

## new model

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

In [13]:
InceptionV3 = InceptionV3(include_top= False, input_shape= input_shape_size, weights= 'imagenet')

87924736/87910968 [==============================] - 1s 0us/step


In [14]:
model = Sequential()

for layer in InceptionV3.layers:
    layer.trainable= False
#     print(layer,layer.trainable)
    
model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))

model.add(Dense(1500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(240,activation='relu'))
model.add(Dense(120,activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              3073500   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1500)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               750500    
_________________________________________________________________
dropout_5 (Dropout)          (None, 500)              

In [15]:
model.compile(optimizer= tf.optimizers.Adam(lr= 0.0001), loss= 'categorical_crossentropy', metrics= ['accuracy'])


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [16]:
from tensorflow.keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='accuracy', 
    patience=8,
    min_delta=0.001
)

In [17]:
model.fit_generator(
    train_generator, 
    epochs = no_of_epochs_2, 
    steps_per_epoch = 128, 
    validation_data= val_generator, 
    shuffle=True,
    callbacks=[custom_early_stopping]
)

Epoch 1/100
128/128 [==============================] - 118s 878ms/step - loss: 4.7429 - accuracy: 0.0296 - val_loss: 4.3563 - val_accuracy: 0.2931
Epoch 2/100
128/128 [==============================] - 109s 851ms/step - loss: 4.1800 - accuracy: 0.1265 - val_loss: 3.2367 - val_accuracy: 0.5186
Epoch 3/100
128/128 [==============================] - 108s 847ms/step - loss: 3.2277 - accuracy: 0.3038 - val_loss: 1.7170 - val_accuracy: 0.6766
Epoch 4/100
128/128 [==============================] - 110s 861ms/step - loss: 2.2555 - accuracy: 0.4797 - val_loss: 1.0992 - val_accuracy: 0.7392
Epoch 5/100
128/128 [==============================] - 109s 854ms/step - loss: 1.6927 - accuracy: 0.5826 - val_loss: 0.8368 - val_accuracy: 0.7774
Epoch 6/100
128/128 [==============================] - 116s 906ms/step - loss: 1.4449 - accuracy: 0.6240 - val_loss: 0.7623 - val_accuracy: 0.7882
Epoch 7/100
128/128 [==============================] - 110s 860ms/step - loss: 1.2719 - accuracy: 0.6607 - val_loss: 0

In [18]:
model.save('inceptv3v1.h5')

In [19]:
import glob

In [20]:
test_imgs = [x for x in glob.glob("../input/dog-breed-identification-mlbootcamp/test/*")]

# training_labels_2 = list(training_set['breed'])
testing_set = pd.DataFrame( {'Images': test_imgs})#,'Breed' : training_labels_2})

#Changing the type  to str
# training_set.Animal = training_set.Images.astype(str)
# testing_set.Images = training_set.Images.astype(str)

In [21]:
len(test_imgs)

10357

In [22]:
testing_set.head()

,Images
0,../input/dog-breed-identification-mlbootcamp/t...
1,../input/dog-breed-identification-mlbootcamp/t...
2,../input/dog-breed-identification-mlbootcamp/t...
3,../input/dog-breed-identification-mlbootcamp/t...
4,../input/dog-breed-identification-mlbootcamp/t...


In [23]:
test_dataGen = ImageDataGenerator(rescale = 1./255
                                  )

test_generator = test_dataGen.flow_from_dataframe(
                                        dataframe = testing_set,
                                        directory="",x_col="Images",
                                        y_col= None,
                                        class_mode=None,
                                        target_size=target_size,
                                        batch_size=32,
                                        
                                        validate_filenames=False)

Found 10357 non-validated image filenames.


In [24]:
test_generator.reset()

In [25]:
pred = model.predict_generator(test_generator)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [26]:
pred.shape

(10357, 120)

In [27]:
labels = (train_generator.class_indices)
labels

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'chesapeake_bay_retriever': 28,
 'chihuahua': 29,
 'chow': 30,
 'clumber': 31,
 'cocker_spaniel': 32,
 'collie': 33,
 'curly-coated_retriever': 34,
 'dandie_dinmont': 35,
 'dhole': 36,
 'dingo': 37,
 'doberman': 38,
 'english_foxhound': 39,
 'english_setter': 40,
 'english_springer': 41,
 'entlebucher': 42,
 'eskimo_dog': 43,
 'flat-coated_retriever': 44,
 'french_bulldog': 45,
 'german_shepherd'

In [28]:
sub = pd.DataFrame(pred, columns= labels)
sub

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,3.965428e-06,3.555413e-10,2.811058e-09,2.158918e-08,2.076661e-06,2.813770e-07,4.042792e-09,1.680653e-07,2.277419e-09,7.045720e-07,...,1.257664e-06,5.733445e-08,4.058716e-09,2.116434e-08,7.398206e-08,3.747504e-09,1.002894e-08,5.572810e-07,1.208934e-08,3.276902e-07
1,5.837840e-09,5.134235e-10,1.153499e-08,1.561745e-08,1.323535e-06,8.431612e-07,1.825780e-12,5.521982e-07,1.306101e-09,1.395120e-07,...,5.760546e-09,1.402305e-07,2.702692e-08,1.988056e-08,2.817403e-07,7.409551e-09,2.845469e-11,2.040459e-02,1.013815e-08,1.941132e-09
2,4.606983e-09,1.080644e-08,4.788212e-09,5.652868e-07,2.100163e-08,2.965148e-07,2.428972e-07,3.322111e-05,2.962409e-07,7.722606e-07,...,4.497360e-06,1.093551e-06,3.159599e-06,1.971289e-07,2.257774e-09,5.380739e-09,1.759136e-07,3.457108e-07,5.673015e-08,4.520470e-09
3,5.039631e-08,1.627890e-07,1.719514e-09,2.195345e-08,2.865362e-08,9.495071e-08,2.363391e-06,6.645515e-08,2.625799e-09,3.296041e-09,...,3.495141e-08,1.789387e-08,8.832123e-10,6.857859e-10,6.750817e-10,1.148031e-09,1.404279e-07,2.768202e-09,5.094814e-07,1.689794e-07
4,1.796903e-06,1.172255e-05,9.225883e-06,4.655650e-06,1.392586e-05,5.278636e-01,1.004533e-04,1.181901e-04,5.909060e-06,9.734429e-05,...,8.920334e-07,9.773927e-05,3.550645e-06,2.048810e-04,2.383701e-05,1.235181e-05,1.498098e-06,2.261992e-04,2.629853e-06,1.097441e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10352,3.961302e-05,1.202342e-05,2.298983e-06,5.520268e-06,8.009777e-07,3.314631e-07,3.505945e-08,1.255825e-07,4.567594e-08,4.178712e-08,...,1.869008e-06,2.607228e-07,2.853082e-08,9.573118e-09,1.141142e-07,3.290336e-07,1.120308e-06,3.488707e-07,3.383655e-04,1.959983e-07
10353,7.221428e-14,8.372241e-07,1.594701e-10,2.250424e-11,1.227431e-10,2.931455e-10,2.839758e-13,7.287186e-12,3.230733e-09,3.361509e-08,...,4.015214e-11,1.507214e-11,5.247152e-12,7.628298e-09,1.155411e-09,8.491267e-10,2.338548e-12,5.220491e-07,1.457928e-10,2.539347e-12
10354,7.374506e-08,2.647847e-11,4.023969e-11,5.722448e-09,1.261836e-06,2.939503e-08,3.334598e-09,3.456638e-08,2.404345e-10,2.765544e-07,...,1.001068e-07,8.584636e-08,3.527252e-10,1.626280e-08,7.834141e-09,1.731134e-10,2.041534e-09,1.284427e-07,6.148948e-09,1.201948e-07
10355,2.585262e-08,5.139227e-10,5.692190e-09,5.769604e-08,1.186171e-06,3.475222e-07,1.242412e-08,1.085963e-05,7.861997e-11,3.605885e-09,...,1.850542e-08,9.998914e-01,2.295237e-10,5.273837e-07,7.326332e-10,1.627272e-10,8.462760e-08,5.660319e-08,7.867575e-06,2.474201e-07


In [29]:
testing_set['Images'] = [i.split('/')[-1].split('.')[0] for i in testing_set['Images']] 

In [30]:
testing_set

,Images
0,09e51e17e2b756ff2ace8a87bd1443fa
1,f7a32505c12649183c5991ecfa7d68b3
2,bc6d50ffb4644feb34530aa58943e85b
3,7cbc041e79135a572aad87904b5c9c57
4,f3b603f10d6a344f0395fb46b242ff8e
...,...
10352,d28a328d59d0b2d37fb3ae7ff0128451
10353,e60f6620bf8751538db9006d425f1b45
10354,54f6834b3819d6f0e9ff966070f35668
10355,0608afccc6d9b117730a2ffdb3314d18


In [31]:
sub.insert(0, 'id', testing_set['Images'])

In [32]:
sub

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,09e51e17e2b756ff2ace8a87bd1443fa,3.965428e-06,3.555413e-10,2.811058e-09,2.158918e-08,2.076661e-06,2.813770e-07,4.042792e-09,1.680653e-07,2.277419e-09,...,1.257664e-06,5.733445e-08,4.058716e-09,2.116434e-08,7.398206e-08,3.747504e-09,1.002894e-08,5.572810e-07,1.208934e-08,3.276902e-07
1,f7a32505c12649183c5991ecfa7d68b3,5.837840e-09,5.134235e-10,1.153499e-08,1.561745e-08,1.323535e-06,8.431612e-07,1.825780e-12,5.521982e-07,1.306101e-09,...,5.760546e-09,1.402305e-07,2.702692e-08,1.988056e-08,2.817403e-07,7.409551e-09,2.845469e-11,2.040459e-02,1.013815e-08,1.941132e-09
2,bc6d50ffb4644feb34530aa58943e85b,4.606983e-09,1.080644e-08,4.788212e-09,5.652868e-07,2.100163e-08,2.965148e-07,2.428972e-07,3.322111e-05,2.962409e-07,...,4.497360e-06,1.093551e-06,3.159599e-06,1.971289e-07,2.257774e-09,5.380739e-09,1.759136e-07,3.457108e-07,5.673015e-08,4.520470e-09
3,7cbc041e79135a572aad87904b5c9c57,5.039631e-08,1.627890e-07,1.719514e-09,2.195345e-08,2.865362e-08,9.495071e-08,2.363391e-06,6.645515e-08,2.625799e-09,...,3.495141e-08,1.789387e-08,8.832123e-10,6.857859e-10,6.750817e-10,1.148031e-09,1.404279e-07,2.768202e-09,5.094814e-07,1.689794e-07
4,f3b603f10d6a344f0395fb46b242ff8e,1.796903e-06,1.172255e-05,9.225883e-06,4.655650e-06,1.392586e-05,5.278636e-01,1.004533e-04,1.181901e-04,5.909060e-06,...,8.920334e-07,9.773927e-05,3.550645e-06,2.048810e-04,2.383701e-05,1.235181e-05,1.498098e-06,2.261992e-04,2.629853e-06,1.097441e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10352,d28a328d59d0b2d37fb3ae7ff0128451,3.961302e-05,1.202342e-05,2.298983e-06,5.520268e-06,8.009777e-07,3.314631e-07,3.505945e-08,1.255825e-07,4.567594e-08,...,1.869008e-06,2.607228e-07,2.853082e-08,9.573118e-09,1.141142e-07,3.290336e-07,1.120308e-06,3.488707e-07,3.383655e-04,1.959983e-07
10353,e60f6620bf8751538db9006d425f1b45,7.221428e-14,8.372241e-07,1.594701e-10,2.250424e-11,1.227431e-10,2.931455e-10,2.839758e-13,7.287186e-12,3.230733e-09,...,4.015214e-11,1.507214e-11,5.247152e-12,7.628298e-09,1.155411e-09,8.491267e-10,2.338548e-12,5.220491e-07,1.457928e-10,2.539347e-12
10354,54f6834b3819d6f0e9ff966070f35668,7.374506e-08,2.647847e-11,4.023969e-11,5.722448e-09,1.261836e-06,2.939503e-08,3.334598e-09,3.456638e-08,2.404345e-10,...,1.001068e-07,8.584636e-08,3.527252e-10,1.626280e-08,7.834141e-09,1.731134e-10,2.041534e-09,1.284427e-07,6.148948e-09,1.201948e-07
10355,0608afccc6d9b117730a2ffdb3314d18,2.585262e-08,5.139227e-10,5.692190e-09,5.769604e-08,1.186171e-06,3.475222e-07,1.242412e-08,1.085963e-05,7.861997e-11,...,1.850542e-08,9.998914e-01,2.295237e-10,5.273837e-07,7.326332e-10,1.627272e-10,8.462760e-08,5.660319e-08,7.867575e-06,2.474201e-07


In [33]:
sub.to_csv('submission1.csv', index = False)